## I. Import neccessay libraries

### A. For PostGreSQL Set Up

In [3]:
from psycopg2.extras import execute_values
from dotenv import load_dotenv
import psycopg2
import os

### B. For Data Retrieve, Process

In [7]:
from urllib.request import urlopen
from datetime import datetime
import polars as pl
import requests
import finnhub
import certifi
import json
import ssl

## II. Set Up Global Variables

### A. For PostGreSQL

In [46]:
load_dotenv("./keys/postgresql.env", override=True)

POSTGRESQL_HOST = os.getenv("POSTGRESQL_HOST")
POSTGRESQL_PORT = os.getenv("POSTGRESQL_PORT")
POSTGRESQL_DB = os.getenv("POSTGRESQL_DB")
POSTGRESQL_USER = os.getenv("POSTGRESQL_USER")
POSTGRESQL_PASSWORD = os.getenv("POSTGRESQL_PASSWORD")

DATABASE_STRUCT_DICT = {
    "bronze_schema" :{"stock_prices_fact_table":{"columns_names": ['stock_symbol', 'current_price', 'change', 'percent_change', 'high_price_of_the_day', 'low_price_of_the_day', 'open_price_of_the_day', 'previous_close_price', 'Datetime'], "columns_data_types_list": ["String", "Float64", "Float64", "Float64", "Float64", "Float64", "Float64", "Float64", "String"]},
                      "company_information_dim_table":{"columns_names": ['symbol', 'price', 'beta', 'volAvg', 'mktCap', 'lastDiv', 'range', 'changes', 'companyName', 'currency', 'cik', 'isin', 'cusip', 'exchange', 'exchangeShortName', 'industry', 'website', 'description', 'ceo', 'sector', 'country', 'fullTimeEmployees', 'phone', 'address', 'city', 'state', 'zip', 'dcfDiff', 'dcf', 'image', 'ipoDate', 'defaultImage', 'isEtf', 'isActivelyTrading', 'isAdr', 'isFund', 'Datetime'], "columns_data_types_list": ["String", "Float64", "Float64", "Int64", "Int64", "Float64", "String", "Float64", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "Float64", "Float64", "String", "String", "Boolean", "Boolean", "Boolean", "Boolean", "Boolean", "String"]}},
    "silver_schema" :{"silver_stock_prices_fact":{"columns_names": ['stock_symbol', 'current_price', 'change', 'percent_change', 'high_price_of_the_day', 'low_price_of_the_day', 'open_price_of_the_day', 'previous_close_price', 'Datetime', 'Date', 'Day', 'Month', 'Year', 'Hour', 'Min', 'Sec'], "columns_data_types_list": ["String", "Float64", "Float64", "Float64", "Float64", "Float64", "Float64", "Float64", "String", "Date", "Int8", "Int8", "Int32", "Int8", "Int8", "Int8"]},
                      "silver_company_info_dim":{"columns_names": ['symbol', 'price', 'beta', 'volAvg', 'mktCap', 'lastDiv', 'range', 'changes', 'companyName', 'currency', 'cik', 'isin', 'cusip', 'exchange', 'exchangeShortName', 'industry', 'website', 'description', 'ceo', 'sector', 'country', 'fullTimeEmployees', 'phone', 'address', 'city', 'state', 'zip', 'dcfDiff', 'dcf', 'image', 'ipoDate', 'defaultImage', 'isEtf', 'isActivelyTrading', 'isAdr', 'isFund', 'Datetime', 'Date', 'Day', 'Month', 'Year', 'Hour', 'Min', 'Sec'], "columns_data_types_list": ["String", "Float64", "Float64", "Int64", "Int64", "Float64", "String", "Float64", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "Float64", "Float64", "String", "String", "Boolean", "Boolean", "Boolean", "Boolean", "Boolean", "String", "Date", "Int8", "Int8", "Int32", "Int8", "Int8", "Int8"]}},
    "gold_schema" :{"gold_table":{"columns_names": ['stock_symbol', 'companyName', 'current_price', 'percent_change', 'mktCap', 'sector', 'industry', 'country', 'Date', 'Day', 'Month', 'Year', 'Hour', 'Min', 'Sec'], "columns_data_types_list": ["String", "String", "Float64", "Float64", "Int64", "String", "String", "String", "Date", "Int8", "Int8", "Int32", "Int8", "Int8", "Int8"]},
                      "gold_company_information_dim_table":{"columns_names": ['symbol', 'price', 'beta', 'volAvg', 'mktCap', 'lastDiv', 'range', 'changes', 'companyName', 'currency', 'cik', 'isin', 'cusip', 'exchange', 'exchangeShortName', 'industry', 'website', 'description', 'ceo', 'sector', 'country', 'fullTimeEmployees', 'phone', 'address', 'city', 'state', 'zip', 'dcfDiff', 'dcf', 'image', 'ipoDate', 'defaultImage', 'isEtf', 'isActivelyTrading', 'isAdr', 'isFund', 'Datetime', 'Date', 'Day', 'Month', 'Year', 'Hour', 'Min', 'Sec'], "columns_data_types_list": ["String", "Float64", "Float64", "Int64", "Int64", "Float64", "String", "Float64", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "String", "Float64", "Float64", "String", "String", "Boolean", "Boolean", "Boolean", "Boolean", "Boolean", "String", "Date", "Int8", "Int8", "Int32", "Int8", "Int8", "Int8"]}}
}

DATA_TYPE_MAPPING_POLARS_TO_SQL = {
    "str": "TEXT",
    "f64": "DOUBLE PRECISION",
    "i64": "BIGINT",
    "bool": "BOOLEAN",
    "String": "TEXT",
    "Float64": "DOUBLE PRECISION",
    "Int8": "SMALLINT",
    "Int32": "INTEGER",
    "Boolean": "BOOLEAN",
    "Date": "DATE",
    "Int64": "BIGINT",
    "Int16": "SMALLINT"
}

### Note Full Format Version for Database Structure Dictionary

### B. For Finhub APIs

In [ ]:
load_dotenv("./keys/finhub.env", override=True)

FINHUB_API = os.getenv("FINHUB_API")
FINANCIALMODELINGPREP_API = os.getenv("FINANCIALMODELINGPREP_API")
FINANCIALMODELINGPREP_PROFILE_ROUTE = os.getenv("FINANCIALMODELINGPREP_PROFILE_ROUTE")
STOCK_LIST_FILE_PATH = os.getenv("STOCK_LIST_FILE_PATH")
STOCK_LIST = []
with open(STOCK_LIST_FILE_PATH, 'r') as file:
    for line in file:
        STOCK_LIST.append(line.strip())

#### Set Up FINHUB API

In [ ]:
finnhub_client = finnhub.Client(api_key=FINHUB_API)

## III. Database Set Up Implementation

#### **Summary of Flow**

1. Initialize the connection parameters.
    - host, port, database, user, and password.
<br>    
2. Establish a connection to the PostgreSQL database.
<br>
3. Create **Bronze, Silver, Gold** schemas if they do not already exist.
<br>
4. Create tables with the specified columns and data types in each schema.
<br>
5. Execute the SQL scripts for creating schemas and tables.
<br>
6. Close the database connection.

In [47]:
class PostgreSQLSetUp:
    def __init__(self, host, port, database, user, password):
        """
        Initialize the PostgreSQLSetUp with connection parameters.
        """
        self.host = host
        self.port = port
        self.database = database
        self.user = user
        self.password = password
        self.connection = None

    def connect(self):
        """
        Establish a connection to the PostgreSQL database.
        """
        try:
            self.connection = psycopg2.connect(
                host=self.host,
                port=self.port,
                database=self.database,
                user=self.user,
                password=self.password
            )
            print("Connection successful!")
        except Exception as error:
            print(f"Error connecting to PostgreSQL: {error}")

    def close(self):
        """
        Close the database connection.
        """
        if self.connection:
            self.connection.close()
            print("Connection closed.")

    def execute_sql(self, sql_script):
        """
        Execute a given SQL script.
        """
        if self.connection is None:
            print("Connection not established. Please call the connect() method first.")
            return

        try:
            cursor = self.connection.cursor()
            cursor.execute(sql_script)
            self.connection.commit()
            print("SQL script executed successfully!")
        except Exception as error:
            print(f"Error executing SQL script: {error}")
        finally:
            cursor.close()

    def create_schema(self, schema):
        """
        Create schema if it does not exist.
        """
        try:
            self.connect()
            sql_script = f"CREATE SCHEMA IF NOT EXISTS {schema};"
            self.execute_sql(sql_script)
            self.close()
            return True
        except Exception as error:
            print(f"Error creating schema: {error}")
            return False

    def create_tables(self, schema, table_name, columns_list, columns_data_types_list):
        """
        Create tables in the specified schema with given columns and data types.
        """
        try:
            self.connect()
            columns_definitions = [
                f"{column_name} {self.get_sql_data_type(data_type)}"
                for column_name, data_type in zip(columns_list, columns_data_types_list)
            ]
            columns_definitions_str = ", ".join(columns_definitions)

            sql_script = f"""
            CREATE TABLE IF NOT EXISTS {schema}.{table_name} (
                {columns_definitions_str}
            );
            """
            self.execute_sql(sql_script)
            self.close()
            print(f"Table {schema}.{table_name} created successfully!")
        except Exception as error:
            print(f"Error creating table {schema}.{table_name}: {error}")

    def get_sql_data_type(self, data_type):
        """
        Map Python data types to PostgreSQL data types.
        """
        return DATA_TYPE_MAPPING_POLARS_TO_SQL.get(data_type, "TEXT")

    def set_up(self):
        try:
            for schema, tables in DATABASE_STRUCT_DICT.items():
                if self.create_schema(schema):
                    for table_name, table_info in tables.items():
                        self.create_tables(
                            schema,
                            table_name,
                            table_info["columns_names"],
                            table_info["columns_data_types_list"]
                        )
            return True
        except Exception as error:
            print(f"Error creating table {schema}.{table_name}: {error}")
            return False

if __name__ == "__main__":
    client = PostgreSQLSetUp(POSTGRESQL_HOST, POSTGRESQL_PORT, POSTGRESQL_DB, POSTGRESQL_USER, POSTGRESQL_PASSWORD)
    result = client.set_up()
    if result:
        print("Success")

Connection successful!
SQL script executed successfully!
Connection closed.
Connection successful!
SQL script executed successfully!
Connection closed.
Table bronze_schema.stock_prices_fact_table created successfully!
Connection successful!
SQL script executed successfully!
Connection closed.
Table bronze_schema.company_information_dim_table created successfully!
Connection successful!
SQL script executed successfully!
Connection closed.
Connection successful!
SQL script executed successfully!
Connection closed.
Table silver_schema.silver_stock_prices_fact created successfully!
Connection successful!
SQL script executed successfully!
Connection closed.
Table silver_schema.silver_company_info_dim created successfully!
Connection successful!
SQL script executed successfully!
Connection closed.
Connection successful!
SQL script executed successfully!
Connection closed.
Table gold_schema.gold_table created successfully!
Connection successful!
SQL script executed successfully!
Connection cl

## IV. Retrieve Raw Data

### A. Quote Real Time Prices for Stock List from Finhub API (Raw)

#### 1. Quote Real Time Prices for Stock List

In [ ]:
symbol_prices_dict = dict()

for symbol in stock_list:
    price_data = finnhub_client.quote(symbol)
    symbol_prices_dict[symbol] = price_data

#### 2. Convert dictionary to list of rows and create polars dataframe

In [ ]:
rows = [
    {
        "stock_symbol": symbol,
        "current_price": data["c"],
        "change": data["d"],
        "percent_change": data["dp"],
        "high_price_of_the_day": data["h"],
        "low_price_of_the_day": data["l"],
        "open_price_of_the_day": data["o"],
        "previous_close_price": data["pc"]
    }
    for symbol, data in symbol_prices_dict.items()
]
df = pl.DataFrame(rows)

In [ ]:
df

### B. Retrieve Company Information from Financial Modeling Prep API (Raw)

#### 1. Retrieve Company Information for Stock List

In [ ]:
symbol_company_infor_dict = dict()

def get_jsonparsed_data(url):
    ssl_context = ssl.create_default_context(cafile=certifi.where())
    response = urlopen(url, context=ssl_context)
    data = response.read().decode("utf-8")
    return json.loads(data)
    
for symbol in stock_list:
    url = (FINANCIALMODELINGPREP_PROFILE_ROUTE + symbol + "?apikey=" + FINANCIALMODELINGPREP_API)
    company_infor = get_jsonparsed_data(url)
    symbol_company_infor_dict[symbol] = company_infor

#### 2. Convert dictionary to list of rows and create polars dataframe

In [ ]:
rows_company_infor_dataframe = [data for symbol, data_list in symbol_company_infor_dict.items() for data in data_list]
df_company_infor = pl.DataFrame(rows_company_infor_dataframe)

In [ ]:
df_company_infor

### Retrieve Raw Data Full Class

In [34]:
class RetrieveRawDataProcess:
    def __init__(self, env_path="./keys/finhub.env"):
        load_dotenv(env_path, override=True)
        self.stock_list = []
        self.symbol_prices_dict = {}
        self.symbol_company_info_dict = {}
        self.stock_list_file_path = os.getenv("STOCK_LIST_FILE_PATH")
        self.set_up_finhub_api()
        self.set_up_financial_modeling_prep_api()

    def set_up_finhub_api(self):
        self.finhub_api_key = os.getenv("FINHUB_API")
        self.finnhub_client = finnhub.Client(api_key=self.finhub_api_key)
        
    def set_up_financial_modeling_prep_api(self):
        self.financialmodelingprep_api_key = os.getenv("FINANCIALMODELINGPREP_API")
        self.profile_route = os.getenv("FINANCIALMODELINGPREP_PROFILE_ROUTE")

    def retrieve_data_process_run(self):
        try:
            self.load_stock_list()
            self.get_stock_prices()
            self.get_company_information()
            return self.to_dataframe()
        except Exception as e:
            print(f"Error retrieving price data for {symbol}: {e}")
            return None, None

    def load_stock_list(self):
        """Load the list of stock symbols from the specified file."""
        with open(self.stock_list_file_path, 'r') as file:
            self.stock_list = [line.strip() for line in file]

    def get_stock_prices(self):
        """Retrieve current stock price data for each symbol in the stock list."""
        for symbol in self.stock_list:
            try:
                price_data = self.finnhub_client.quote(symbol)
                self.symbol_prices_dict[symbol] = price_data
            except Exception as e:
                print(f"Error retrieving price data for {symbol}: {e}")

    def get_company_information(self):
        """Fetch company information for each stock symbol using the Financial Modeling Prep API."""
        for symbol in self.stock_list:
            try:
                url = f"{self.profile_route}{symbol}?apikey={self.financialmodelingprep_api_key}"
                response = requests.get(url)
                response.raise_for_status()
                company_info = response.json()
                self.symbol_company_info_dict[symbol] = company_info
            except Exception as e:
                print(f"Error retrieving company information for {symbol}: {e}")

    def to_dataframe(self):
        """Convert the collected data to Polars DataFrames and add a Datetime column."""
        current_time = datetime.now().isoformat()

        # Stock prices DataFrame
        stock_rows = [
            {
                "stock_symbol": symbol,
                "current_price": data.get("c"),
                "change": data.get("d"),
                "percent_change": data.get("dp"),
                "high_price_of_the_day": data.get("h"),
                "low_price_of_the_day": data.get("l"),
                "open_price_of_the_day": data.get("o"),
                "previous_close_price": data.get("pc"),
                "Datetime": current_time
            }
            for symbol, data in self.symbol_prices_dict.items()
        ]
        df_stock_prices = pl.DataFrame(stock_rows)

        # Company information DataFrame
        company_rows = [
            {
                **data,
                "Datetime": current_time
            }
            for symbol, data_list in self.symbol_company_info_dict.items() for data in data_list
        ]
        df_company_info = pl.DataFrame(company_rows)

        return df_stock_prices, df_company_info

if __name__ == "__main__":
    process = RetrieveRawDataProcess()
    df_stock_prices, df_company_info = process.retrieve_data_process_run()
    print("Stock Prices DataFrame:")
    print(df_stock_prices)
    print("\nCompany Information DataFrame:")
    print(df_company_info)

Stock Prices DataFrame:
shape: (5, 9)
┌──────────────┬───────────────┬────────┬────────────────┬───┬──────────────────────┬───────────────────────┬──────────────────────┬────────────────────────────┐
│ stock_symbol ┆ current_price ┆ change ┆ percent_change ┆ … ┆ low_price_of_the_day ┆ open_price_of_the_day ┆ previous_close_price ┆ Datetime                   │
│ ---          ┆ ---           ┆ ---    ┆ ---            ┆   ┆ ---                  ┆ ---                   ┆ ---                  ┆ ---                        │
│ str          ┆ f64           ┆ f64    ┆ f64            ┆   ┆ f64                  ┆ f64                   ┆ f64                  ┆ str                        │
╞══════════════╪═══════════════╪════════╪════════════════╪═══╪══════════════════════╪═══════════════════════╪══════════════════════╪════════════════════════════╡
│ AAPL         ┆ 228.02        ┆ 3.02   ┆ 1.3422         ┆ … ┆ 225.17               ┆ 225.25                ┆ 225.0                ┆ 2024-11-18T17:53:36

### V. Processing Data For Silver, Gold Table

In [49]:
class DataProcessingClass:
    def __init__(self, stock_prices_df: pl.DataFrame, company_info_df: pl.DataFrame):
        self.stock_prices_df = stock_prices_df
        self.company_info_df = company_info_df
        self.silver_stock_prices_df = None
        self.silver_company_info_df = None
        self.gold_table_df = None

    @staticmethod
    def clean_data(df: pl.DataFrame) -> pl.DataFrame:
        """Remove rows with null values from the DataFrame."""
        return df.drop_nulls()

    @staticmethod
    def extract_datetime_components(df: pl.DataFrame, datetime_column: str = "Datetime") -> pl.DataFrame:
        """Extract Date, Day, Month, Year, Hour, Min, Sec from Datetime."""
        df = df.with_columns(
            pl.col(datetime_column).cast(pl.Utf8).str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%6f").alias("ParsedDatetime")
        ).with_columns([
            pl.col("ParsedDatetime").dt.date().alias("Date"),
            pl.col("ParsedDatetime").dt.day().alias("Day"),
            pl.col("ParsedDatetime").dt.month().alias("Month"),
            pl.col("ParsedDatetime").dt.year().alias("Year"),
            pl.col("ParsedDatetime").dt.hour().alias("Hour"),
            pl.col("ParsedDatetime").dt.minute().alias("Min"),
            pl.col("ParsedDatetime").dt.second().alias("Sec")
        ]).drop("ParsedDatetime")
        return df

    def prepare_silver_tables(self):
        """Clean and transform stock prices and company info DataFrames into Silver tables."""
        self.silver_stock_prices_df = self.clean_data(self.stock_prices_df)
        self.silver_stock_prices_df = self.extract_datetime_components(self.silver_stock_prices_df)

        self.silver_company_info_df = self.clean_data(self.company_info_df)
        self.silver_company_info_df = self.extract_datetime_components(self.silver_company_info_df)

    def create_gold_table(self):
        """Combine Silver tables to create a Gold table."""
        # Merge Silver tables on the stock symbol
        self.gold_table_df = self.silver_stock_prices_df.join(
            self.silver_company_info_df,
            left_on="stock_symbol",
            right_on="symbol",
            how="inner"
        )
        self.gold_table_df = self.gold_table_df.select([
            "stock_symbol",
            "companyName",
            "current_price",
            "percent_change",
            "mktCap",
            "sector",
            "industry",
            "country",
            "Date",
            "Day",
            "Month",
            "Year",
            "Hour",
            "Min",
            "Sec"
        ])

    def process_raw_data_and_retrieve_silver_tables(self):
        """Full data processing pipeline."""
        self.prepare_silver_tables()
        self.create_gold_table()
        return self.get_silver_stock_prices(), self.get_silver_company_info()

    def get_silver_stock_prices(self) -> pl.DataFrame:
        """Retrieve the Silver table for stock prices."""
        return self.silver_stock_prices_df

    def get_silver_company_info(self) -> pl.DataFrame:
        """Retrieve the Silver table for company information."""
        return self.silver_company_info_df

    def get_gold_table(self) -> pl.DataFrame:
        """Retrieve the Gold table."""
        return self.gold_table_df

if __name__ == "__main__":
    print("Bronze Stock Prices DataFrame:")
    print("\nBronze Stock Prices Columns: " + str(df_stock_prices.columns) + "\n")
    print("\nBronze Stock Prices Types: " + str(df_stock_prices.dtypes) + "\n")
    print(df_stock_prices)
    
    print("\nBronze Company Info DataFrame:")
    print("\nBronze Company Info Columns: " + str(df_company_info.columns) + "\n")
    print("\nBronze Company Info Types: " + str(df_company_info.dtypes) + "\n")
    print(df_company_info)
    
    processor = DataProcessingClass(df_stock_prices, df_company_info)
    silver_stock_prices, silver_company_info = processor.process_raw_data_and_retrieve_silver_tables()
    gold_table = processor.get_gold_table()
    
    print("\nSilver Stock Prices DataFrame:")
    print("\nSilver Stock Prices Columns: " + str(silver_stock_prices.columns) + "\n")
    print("\nSilver Stock Prices Types: " + str(silver_stock_prices.dtypes) + "\n")
    print(silver_stock_prices)
    
    print("\nSilver Company Info DataFrame:")
    print("\nSilver Company Info Columns: " + str(silver_company_info.columns) + "\n")
    print("\nSilver Company Info Types: " + str(silver_company_info.dtypes) + "\n")
    print(silver_company_info)
    
    print("\nGold Table DataFrame:")
    print("\nGold Columns: " + str(gold_table.columns) + "\n")
    print("\nnGold Types: " + str(gold_table.dtypes) + "\n")
    print(gold_table)

Bronze Stock Prices DataFrame:

Bronze Stock Prices Columns: ['stock_symbol', 'current_price', 'change', 'percent_change', 'high_price_of_the_day', 'low_price_of_the_day', 'open_price_of_the_day', 'previous_close_price', 'Datetime']


Bronze Stock Prices Types: [String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, String]

shape: (5, 9)
┌──────────────┬───────────────┬────────┬────────────────┬───┬──────────────────────┬───────────────────────┬──────────────────────┬────────────────────────────┐
│ stock_symbol ┆ current_price ┆ change ┆ percent_change ┆ … ┆ low_price_of_the_day ┆ open_price_of_the_day ┆ previous_close_price ┆ Datetime                   │
│ ---          ┆ ---           ┆ ---    ┆ ---            ┆   ┆ ---                  ┆ ---                   ┆ ---                  ┆ ---                        │
│ str          ┆ f64           ┆ f64    ┆ f64            ┆   ┆ f64                  ┆ f64                   ┆ f64                  ┆ str                     

In [68]:
class LoadDataProcess:
    def __init__(self):
        load_dotenv("./keys/postgresql.env", override=True)
        self.postgresql_config = {
            "host": os.getenv("POSTGRESQL_HOST"),
            "port": os.getenv("POSTGRESQL_PORT"),
            "dbname": os.getenv("POSTGRESQL_DB"),
            "user": os.getenv("POSTGRESQL_USER"),
            "password": os.getenv("POSTGRESQL_PASSWORD"),
        }
        self.db_struct = DATABASE_STRUCT_DICT
        self.type_mapping = DATA_TYPE_MAPPING_POLARS_TO_SQL

    def connect_to_db(self):
        try:
            conn = psycopg2.connect(**self.postgresql_config)
            return conn
        except Exception as e:
            raise Exception(f"Error connecting to PostgreSQL: {e}")

    def load_data_to_table(self, conn, schema, table_name, data):
        """
        Insert data into the specified table using execute_values
        """
        with conn.cursor() as cur:
            columns = data.columns
            table_columns = ", ".join(columns)
            query = f"INSERT INTO {schema}.{table_name} ({table_columns}) VALUES %s"
            data_values = [tuple(row) for row in data.to_numpy()]
            execute_values(cur, query, data_values)
            conn.commit()

    def process_table(self, schema, table_name, data, conn):
        """
        Process a table: Create and load data
        """
        table_info = self.db_struct[schema][table_name]
        self.load_data_to_table(conn, schema, table_name, data)

    def load_all_data(self, dataframes):
        """
        Load all DataFrames into their respective tables
        """
        conn = self.connect_to_db()
        try:
            for schema, tables in self.db_struct.items():
                for table_name in tables.keys():
                    if table_name in dataframes:
                        self.process_table(schema, table_name, dataframes[table_name], conn)
        finally:
            conn.close()

dataframes = {
    "stock_prices_fact_table": df_stock_prices,
    "company_information_dim_table": df_company_info,
    "silver_stock_prices_fact": silver_stock_prices,
    "silver_company_info_dim": silver_company_info,
    "gold_table": gold_table,
    "gold_company_information_dim_table": silver_company_info,
}
loader = LoadDataProcess()
loader.load_all_data(dataframes)

/n
INSERT INTO bronze_schema.stock_prices_fact_table (stock_symbol, current_price, change, percent_change, high_price_of_the_day, low_price_of_the_day, open_price_of_the_day, previous_close_price, Datetime) VALUES %s
/n
[('AAPL', 228.02, 3.02, 1.3422, 229.74, 225.17, 225.25, 225.0, '2024-11-18T17:53:36.747774'), ('MSFT', 415.76, 0.76, 0.1831, 418.4037, 412.1, 414.87, 415.0, '2024-11-18T17:53:36.747774'), ('GOOGL', 175.3, 2.81, 1.6291, 175.438, 172.9, 173.42, 172.49, '2024-11-18T17:53:36.747774'), ('AMZN', 201.7, -0.91, -0.4491, 204.67, 200.95, 204.15, 202.61, '2024-11-18T17:53:36.747774'), ('TSLA', 338.74, 18.02, 5.6186, 348.5499, 330.01, 340.73, 320.72, '2024-11-18T17:53:36.747774')]
/n
INSERT INTO bronze_schema.company_information_dim_table (symbol, price, beta, volAvg, mktCap, lastDiv, range, changes, companyName, currency, cik, isin, cusip, exchange, exchangeShortName, industry, website, description, ceo, sector, country, fullTimeEmployees, phone, address, city, state, zip, dcfDiff